In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.3 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images, train_labels),(test_images,test_lables) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
          kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
          activation = 'relu',
          input_shape = (28,28,1)
      ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter', min_value=32, max_value=64, step=16),
          kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
          activation = 'relu',
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation = 'relu'
      ),
      keras.layers.Dense(10, activation='softmax') #output Layer
  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('Learning_rate', values = [1e-2,1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [11]:
tuner_search = RandomSearch(build_model,
                            objective = 'val_accuracy',
                            max_trials = 5,directory = 'output', project_name = "Minst Fashion")

In [ ]:
tuner_search.search(train_images, train_labels, epochs = 3, validation_split=0.1)


Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
96                |64                |conv_1_filter
5                 |3                 |conv_2_kernel
128               |96                |dense_1_units
0.001             |0.001             |Learning_rate

Epoch 1/3
1688/1688 [==============================] - 812s 481ms/step - loss: 0.4087 - accuracy: 0.8517 - val_loss: 0.3000 - val_accuracy: 0.8857
Epoch 2/3
 580/1688 [=========>....................] - ETA: 8:44 - loss: 0.2739 - accuracy: 0.8997

In [20]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 96)                3539040   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 3,577,578
Trainable params: 3,577,578
Non-trainable params: 0
_________________________________________________________________
